In [1]:

import os
os.environ['P8_PG_HOST'] = 'localhost'
os.environ['P8_PG_PORT'] = '25432'
os.environ['P8_PG_PASSWORD'] = os.environ.get('P8_TEST_BEARER_TOKEN')
import nest_asyncio
nest_asyncio.apply()

In [2]:
import sys
from percolate.models.p8.types import Resources

sys.path.append('/Users/sirsh/code/res/one-core')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/polars/_cpu_check.py:239: RuntimeWarning: Missing required CPU features.

The following required CPU features were not detected:
    avx, fma
Continuing to use this version of Polars on this processor will likely result in a crash.
Install the `polars-lts-cpu` package instead of `polars` to run Polars with better compatibility.

Hint: If you are on an Apple ARM machine (e.g. M1) this is likely due to running Python under Rosetta.
It is recommended to install a native version of Python that does not run under Rosetta x86-64 emulation.

If you believe this warning to be a false positive, you can set the `POLARS_SKIP_CPU_CHECK` environment variable to bypass this check.

  warnings.warn(


In [3]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/INST_018.wav",
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False,
    
)

len(resources)

2025-05-22 13:17:51.624 | INFO     | percolate.services.FileSystemService:chunk_resource_from_uri:1093 - Chunking resource from URI: /Users/sirsh/Downloads/INST_018.wav (mode: extended)
2025-05-22 13:17:52.271 | INFO     | percolate.services.FileSystemService:_chunk_media_resource:1220 - Processing audio file: /Users/sirsh/Downloads/INST_018.wav
2025-05-22 13:17:52.389 | INFO     | percolate.services.FileSystemService:_chunk_media_resource:1234 - Audio file size: 80.5MB
2025-05-22 13:17:52.390 | INFO     | percolate.services.FileSystemService:_chunk_media_resource:1245 - File size (80.5MB) exceeds limit, chunking audio
2025-05-22 13:17:52.410 | INFO     | percolate.services.FileSystemService:_chunk_large_audio_file:1378 - Loading audio file for chunking: /var/folders/tp/cbc5g2n10qgb0lt3tg00dq2h0000gn/T/tmpec03jmmc.wav
2025-05-22 13:17:53.186 | INFO     | percolate.services.FileSystemService:_chunk_large_audio_file:1385 - Audio duration: 439.6s, file size: 80.5MB
2025-05-22 13:17:53.188

5

In [1]:

# Simple chunking - basic text extraction (fast, no API costs)
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/6-Month Roadmap & Milestones.pdf",
    parsing_mode="simple",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False
)

len(resources)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/polars/_cpu_check.py:239: RuntimeWarning: Missing required CPU features.

The following required CPU features were not detected:
    avx, fma
Continuing to use this version of Polars on this processor will likely result in a crash.
Install the `polars-lts-cpu` package instead of `polars` to run Polars with better compatibility.

Hint: If you are on an Apple ARM machine (e.g. M1) this is likely due to running Python under Rosetta.
It is recommended to install a native version of Python that does not run under Rosetta x86-64 emulation.

If you believe this warning to be a false positive, you can set the `POLARS_SKIP_CPU_CHECK` environment variable to bypass this check.

  warnings.warn(
2025-05-22 13:02:40.839 | INFO     | percolate.services.FileSystemService:chunk_resource_from_uri:1093 - Chunking resource from URI: /Users/sirsh/Downloads/6-Month Roadmap & Milestones.pdf (mode: simple)
2025-05-22 13:02:40.84

56

In [2]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/Transit Insurance Form.pdf", 
    parsing_mode="extended",
    save_to_db=True
)
resources

2025-05-22 13:02:45.131 | INFO     | percolate.services.FileSystemService:chunk_resource_from_uri:1093 - Chunking resource from URI: /Users/sirsh/Downloads/Transit Insurance Form.pdf (mode: extended)
2025-05-22 13:02:45.133 | INFO     | percolate.services.FileSystemService:read:574 - Reading file: /Users/sirsh/Downloads/Transit Insurance Form.pdf
2025-05-22 13:02:47.933 | INFO     | percolate.services.FileSystemService:_convert_pdf_pages_to_images:849 - Converting PDF pages to images using pdf2image convert_from_path for Transit Insurance Form.pdf
2025-05-22 13:02:49.872 | INFO     | percolate.services.FileSystemService:_convert_pdf_pages_to_images:851 - Successfully converted 7 pages to images using convert_from_path
2025-05-22 13:02:49.877 | INFO     | percolate.services.FileSystemService:_extract_extended_pdf_content:1497 - Analyzing 7 PDF pages with LLM vision
2025-05-22 13:02:51.066 | INFO     | percolate.services.llm.ImageInterpreter:describe_images:109 - Describing 1 images usin

[Resources(id='5b5608c6-c350-5e24-a6b7-e4a80d4c3a86', name='Transit Insurance Form.pdf_chunk_1', category='pdf_chunk', content='DOCUMENT ANALYSIS SUMMARY:\n- Document: Transit Insurance Form.pdf\n- Total Pages: 7\n- Analysis Method: LLM Vision + Text Extraction\n- Pages Successfully Analyzed: 7\n\nFULL CONTENT:\n=== PAGE 1 ===\n1. **Text Content Transcription:**\n\n   **Header:**\n   - WTI Logo\n   - "WORLDWIDE TRANSIT INSURANCE BROKERS, INC"\n   - "INTERNATIONAL PROTECTION PLAN - HOUSEHOLD GOODS PROPOSAL FORM"\n\n   **Personal Information:**\n   - Name: saoirse amarteifio\n   - Origin: New York\n   - Email: amartey@gmail.com\n   - Lump sum Amount: $5000.00\n\n   **Conveyance Information:**\n   - Conveyance: Sea\n   - Destination: London\n   - Type of Valuation: Platinum Plan (Lump Sum)\n\n   **Sections with Tables:**\n\n   **Section A - Living Room:**\n   - ARTICLE: CHAIR(S), END TABLE(S), COFFEE TABLE, etc.\n   - QTY and VALUE $ columns\n\n   **Section B - Bathrooms:**\n   - ARTICLE:

In [ ]:
resources = Resources.chunked_resource(
    uri="/path/to/document.pdf", 
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
"""integration test"""

In [3]:
from one.services.ask_one import AgentCallingContext
from one.services.slack.models import SlackMessage
from percolate.utils import make_uuid
def _get_userid_from_username(name:str):
    """map the user id from the email address or slack name"""
    from percolate.services import PostgresService
    pg = PostgresService()
    Q = f"""
            SELECT * FROM p8."User" where LOWER(email) = LOWER(%s) or LOWER(slack_id) = LOWER(%s) limit 1
    """
    
    data = pg.execute(Q, data=(name, name))
    if data:
        return data[0]['id']
    
    
channel_ts = f"123445"
message = SlackMessage(user='U01JDKSB196', 
                       text='hi, can you tell me thre things about resonance', 
                       thread_ts="123454", 
                       ts=channel_ts, 
                       channel='C08TE6NNY7L')
"""TODO think about this session id interface"""
ctx = AgentCallingContext(
    channel_context=message.channel,
    streaming_callback=None,#,buffer.add_data,
    username=message.user,

    session_id=make_uuid( message.thread_ts or message.ts), #the slack message thread is used as a session id - when there is no message thread_ts its because we are the originator
    channel_ts=str(channel_ts),
    session_context=f"{message.channel}-{message.thread_ts or message.ts}",
    s3_files=None,
    process_files_async=True,
    user_id=_get_userid_from_username(message.user)
    
)
setattr(ctx,'max_iterations',5)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/polars/_cpu_check.py:239: RuntimeWarning: Missing required CPU features.

The following required CPU features were not detected:
    avx, fma
Continuing to use this version of Polars on this processor will likely result in a crash.
Install the `polars-lts-cpu` package instead of `polars` to run Polars with better compatibility.

Hint: If you are on an Apple ARM machine (e.g. M1) this is likely due to running Python under Rosetta.
It is recommended to install a native version of Python that does not run under Rosetta x86-64 emulation.

If you believe this warning to be a false positive, you can set the `POLARS_SKIP_CPU_CHECK` environment variable to bypass this check.

  warnings.warn(


In [6]:
import percolate as p8
from percolate.models import PercolateAgent

p8.Agent(PercolateAgent).run('what is a fashion brand',context=ctx)

2025-05-21 12:09:13.437 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function help
2025-05-21 12:09:13.517 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function get_entities
2025-05-21 12:09:13.520 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function search
2025-05-21 12:09:13.521 | DEBUG    | percolate.services.FunctionManager:add_function:40 - added function activate_functions_by_name
2025-05-21 12:09:13.522 | INFO     | percolate.services.ModelRunner:__init__:57 - ******Constructed agent p8.PercolateAgent******
2025-05-21 12:09:13.831 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:320 - invoking model gpt-4.1-2025-04-14, is_streaming=False
2025-05-21 12:09:16.902 | DEBUG    | percolate.services.llm.LanguageModel:__call__:198 - response=<Response [200]>, context=AgentCallingContext(session_id='07f67440-4201-554f-bd28-dc32b652c0e5', session_context='C08TE6NNY7L-123454', prefer_json=Fal

'A fashion brand is a company or label that designs, produces, and sells clothing, accessories, footwear, or other items related to personal style and appearance. Fashion brands often have a distinct identity, aesthetic, or philosophy that sets them apart from others in the industry. They may focus on specific markets, such as luxury, streetwear, sportswear, or fast fashion, and use branding, marketing, and design to create a recognizable image and attract customers.\n\nExamples of well-known fashion brands include Gucci, Nike, Zara, and Louis Vuitton. Each of these brands has its own unique style, target audience, and reputation in the fashion world.'

In [ ]:
#!pip install -U boto3

In [ ]:
from percolate.services import S3Service

s =S3Service()
s

In [ ]:
from io import BytesIO
with open("/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf", 'rb') as f:
    data = BytesIO(f.read())
    data.seek(0)
    s.upload_filebytes_to_uri("s3://percolate/test/file.pdf", data)

In [ ]:
s.upload_file_to_uri("s3://percolate/test/file.pdf",
                     "/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf" )

In [ ]:
import percolate as p8
from percolate.models.media.tus import register_tus_models
from percolate.models import User
#p8.repository(User).register()
#register_tus_models()
from percolate.services import PostgresService

pg = PostgresService()
 

In [ ]:
pg.execute(""" select * from p8."Resources" order by updated_at desc limit 5 """)

In [ ]:
import requests

session = requests.Session()
session.cookies.set('session', 'eyJzeW5jX2ZpbGVzIjogZmFsc2UsICJ0b2tlbiI6IHsiYWNjZXNzX3Rva2VuIjogInlhMjkuYTBBVzRYdHhocFU5Q1ZwU284R2FwR0JQaHZYcHA0S1BJdGREbW9ocmc3ZFNBbllXdEN4ajM3N2pRcTA1Sk03UnhsR3phSk05cWNkU09SWk5rdjlxckdNV1NfUkVqRFhGMVU1Z3FtN0ZDbEhUb3RUM1N3SldyUmJCZGd3d2N2SWlKbWZmSzZDSWZDemdYNHdJZTQ0ZDFlVFV2emkzc1Y4Z2FTTkstX2tENDVhQ2dZS0FXUVNBUmNTRlFIR1gyTWlHLXRJNXlmYzBFTTA5MnFTMnN6aFpnMDE3NSIsICJleHBpcmVzX2luIjogMzU5OSwgInJlZnJlc2hfdG9rZW4iOiAiMS8vMDVZWDNsZG04WXVJeUNnWUlBUkFBR0FVU053Ri1MOUlyYW1wNHlKR2Y5TnA0UW9aUlVGYkdQeVJXaTlWRHZUX0VLdVJ0cFJ3YnJUcHYxS1FVY1VtOFlBV1hQWURBNnhkckc0TSIsICJzY29wZSI6ICJodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL2RyaXZlLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvdXNlcmluZm8ucHJvZmlsZSBodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL3VzZXJpbmZvLmVtYWlsIGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZG9jdW1lbnRzLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZ21haWwucmVhZG9ubHkgb3BlbmlkIiwgInRva2VuX3R5cGUiOiAiQmVhcmVyIiwgImlkX3Rva2VuIjogImV5SmhiR2NpT2lKU1V6STFOaUlzSW10cFpDSTZJalkyTUdWbU0ySTVOemcwWW1SbU5UWmxZbVU0TlRsbU5UYzNaamRtWWpKbE9HTXhZMlZtWm1JaUxDSjBlWEFpT2lKS1YxUWlmUS5leUpwYzNNaU9pSm9kSFJ3Y3pvdkwyRmpZMjkxYm5SekxtZHZiMmRzWlM1amIyMGlMQ0poZW5BaU9pSXhNRGs1TWpNM09EazBPVE0xTFhBemJqZG5hMncyY3prM2FHb3lNRGh1YjNGdE5ISm9PSEpyTTJkaVluWjBMbUZ3Y0hNdVoyOXZaMnhsZFhObGNtTnZiblJsYm5RdVkyOXRJaXdpWVhWa0lqb2lNVEE1T1RJek56ZzVORGt6TlMxd00yNDNaMnRzTm5NNU4yaHFNakE0Ym05eGJUUnlhRGh5YXpOblltSjJkQzVoY0hCekxtZHZiMmRzWlhWelpYSmpiMjUwWlc1MExtTnZiU0lzSW5OMVlpSTZJakV3TWpFMk5UUTNOakV4TkRnd05EQTVOakl6TUNJc0ltVnRZV2xzSWpvaVlXMWhjblJsZVVCbmJXRnBiQzVqYjIwaUxDSmxiV0ZwYkY5MlpYSnBabWxsWkNJNmRISjFaU3dpWVhSZmFHRnphQ0k2SW5OclYyWktjbTVsTlRaMmEyaFRMVnBKVEhOSlNXY2lMQ0p1YjI1alpTSTZJbkp2WlRNM1UzWkJjbmMyU0VwSVVsQTFTbTl0SWl3aWJtRnRaU0k2SWxOaGIybHljMlVnUVcxaGNuUmxhV1pwYnlJc0luQnBZM1IxY21VaU9pSm9kSFJ3Y3pvdkwyeG9NeTVuYjI5bmJHVjFjMlZ5WTI5dWRHVnVkQzVqYjIwdllTOUJRMmM0YjJOTWFWbFFNM0J0TFhSNmFITldURkp1T1ZoaVoxQlFWa2g0U0RCYWVUQmpSMmROZERKcVlVVkJZMnR3VDBsdlRGUkVaejF6T1RZdFl5SXNJbWRwZG1WdVgyNWhiV1VpT2lKVFlXOXBjbk5sSWl3aVptRnRhV3g1WDI1aGJXVWlPaUpCYldGeWRHVnBabWx2SWl3aWFXRjBJam94TnpRM05EQTJPVGszTENKbGVIQWlPakUzTkRjME1UQTFPVGQ5LmM5NWQ5WGtWREFQcjdjU3hBUWVtZzFYWXROSjREcHJhS2JIYlUwb0t1VmNlcDdybTNxQnRqOTZ4c0VnSWc5SXBWVlQ5Ym4zTDF2alU5OTYyVDdQeC1BWEdNYzRab25YX3FYTGc2M1pueGhBVHNIcXdoQ2VoS2N5NVZxYkxkbVVtYnlsZmFYR0pIbFp5ZDhRbkU1RVppY3B1d3ZhN1U5Y3F2Q184ZTVERkN6Ymk5d0V2bHNmc21Vb3U3U0dWdERkX2dqVUxfNTY4RldhSktrNmt3aUhBZXRUaEQ5WmZUaGgybE1vZHdrVHhuOXVzTWhNZ2oyeENvLXR4bjdqa3hRNnk1NGpINE1DRnhUZzk4V2lRdFJvWkxUbm9neXVYbGZzVkI3V3FQdllqTjlrR2tmMnU3SzJCOHFuMkJGT1NjTFJPaHRGSzJDcTIzVm1WZzhNMmVXQlB6dyIsICJyZWZyZXNoX3Rva2VuX2V4cGlyZXNfaW4iOiA2MDQ3OTksICJleHBpcmVzX2F0IjogMTc0NzQxMDU5NiwgInVzZXJpbmZvIjogeyJpc3MiOiAiaHR0cHM6Ly9hY2NvdW50cy5nb29nbGUuY29tIiwgImF6cCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgImF1ZCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgInN1YiI6ICIxMDIxNjU0NzYxMTQ4MDQwOTYyMzAiLCAiZW1haWwiOiAiYW1hcnRleUBnbWFpbC5jb20iLCAiZW1haWxfdmVyaWZpZWQiOiB0cnVlLCAiYXRfaGFzaCI6ICJza1dmSnJuZTU2dmtoUy1aSUxzSUlnIiwgIm5vbmNlIjogInJvZTM3U3ZBcnc2SEpIUlA1Sm9tIiwgIm5hbWUiOiAiU2FvaXJzZSBBbWFydGVpZmlvIiwgInBpY3R1cmUiOiAiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jTGlZUDNwbS10emhzVkxSbjlYYmdQUFZIeEgwWnkwY0dnTXQyamFFQWNrcE9Jb0xURGc9czk2LWMiLCAiZ2l2ZW5fbmFtZSI6ICJTYW9pcnNlIiwgImZhbWlseV9uYW1lIjogIkFtYXJ0ZWlmaW8iLCAiaWF0IjogMTc0NzQwNjk5NywgImV4cCI6IDE3NDc0MTA1OTd9fSwgInNlc3Npb25faWQiOiAiYzU4ZmQ1MWYtNDAzZC00MjU3LTllMTgtMTc3MzdlNDRmZDNmIn0=.aCdQlQ.qHKMWsxfHuBLHnk3Ck3Ux_tiqp8')

response = session.get('http://localhost:5008/auth/ping')
print(response.json())

response = session.get('http://localhost:5008/auth/session/info')
print(response.json())

In [ ]:
session.get('http://localhost:5008/tus/user/uploads?query=anything').json()

In [ ]:
response = session.get('http://localhost:5008/auth/me')
print(response.json())

In [ ]:
from percolate.api.routes.auth.utils import get_user_from_email, is_valid_token_for_user, extract_user_info_from_token
extract_user_info_from_token('eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6ImlBS2ZoVXRMU1ZDbkR4aVE1X3NwUUEiLCJub25jZSI6IjhGamhCTE5TYzBFeDJ2Qm5KSVA1IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4Mjk0LCJleHAiOjE3NDc2MDE4OTR9.P0goYoml9dxdPHiH-f7IjeEFWMv6qDT6cv-GxHma2drk14pkIuwwdU0w2OCDSlFRGc4hfePIvMU3KIddNPGnkc8qvBKDNvafdBkndMQ0D1rae8BNScVr3EuZiikv_O54Eff1Lv6cmcKd978m5I7p8xVx-2aGGrxGELz3OFszgnh6POo1efiuznp6sPIQJbskfI1a8YR1_GArU9jkwhjhczEsibWNtLn5hIzox02qBTdjfrh69eOOKSmysb2SjWl54AmWxubYfR1M1IO0CAPHUQFzVrdouUsBvZnIk3xWnF-roSMrt68aoWWCtzAxtupzy3ziY8RdTVyWy7uEnr-k6g')

In [ ]:
import requests

session = requests.Session()
#session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6InNlN2FuOVhHd19BeEVGdkFOSTZSckEiLCJub25jZSI6InZWRlVBekpTU3RsMGZmQnp3RXBsIiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTkzNjk4LCJleHAiOjE3NDc1OTcyOTh9.Z61fPlHZ__DZ38rhlU-PmloMjfs6Tj2xLFqgLj5r2yOHsPDUSEmBYm8Ay7qEmUj5kWV-iyHRgW1c7Z-p3z1k8KUyPVsKXXbZlHvlHUDBmTQo8Uy3axHyei02Vz2ejImBspjlLzjg1bf_wQUXOo5pAdR8D0elaWoRLseslRvyLdZf3k2mk_pxy3c3utkefco1psIaGcdRC44OIdqCCWtPh1MXsSu0fMW4vsdpJQYRrIlmodMWwTLph6Rv2jJ6Rl5JQU3OG6Bx9dW5XQld1k1eQuP2_UEij4-lo3_JsJD4zdbFDSAbiNUbZICof6tatbsDiCpHd3o9-AV_BBlcn_lKdA')

session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6Im5KcnJIMm9lanZvSHRrSUJyS1FLckEiLCJub25jZSI6Imc1bVk5Z3AyaVduSDF6Q2U2MFl5IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4NTcwLCJleHAiOjE3NDc2MDIxNzB9.NpUvgWfPSpQLaQL8UR1mr2Vd4S7-3ZLEoeK2Dbpbi_uBES51dm8MUFnDTQPNULwo-IVUE9GBAr5HInqTx6L95cTiAohTUG-Qzio96rwtnNYF0YWkBByhTKWRKhdQ6YK97TwGffZMrEzaQ_QvHe_XF17exJpKkRLtjijCuMmmAMMqbdgtsd5XVtxSma_HFapIi4dTcZOiVqWglzsGCXWxvZqtWXiHii1niKFYb95aF8q4sRgsRGs4Z7cCc1l6YzD8sSnkTPyay_qWuAWvMlS3iY9x3777zG82U9Fi38sZ2IaQ7CS1IbZ7CrBo45-2iuh28PSw7H0v2qGcTwfOF0lhTA')

response = session.get('https://eepis.percolationlabs.ai/auth/ping')
print(response.json())

response = session.get('https://eepis.percolationlabs.ai/auth/session/info')
print(response.json())

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?limit=10').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=CevolvingAnyibodies.').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=Eta Carinae pulsed in the distance.').json()